<a href="https://colab.research.google.com/github/kuaci1/Homework_3_DataFrame/blob/main/Homework_3_DataFrame_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 – DataFrame
## Data Science Batch 57

---

**Objectives:**
1. Create "Full Name" column with title case
2. Calculate GMV (Gross Merchandise Value)
3. Group categories and create pivot table
4. Find seller with highest GMV in August 2017
5. Find seller with most transactions in Fashion (September 2017)

## Step 1: Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np

# Load the CSV files
paid_transaction = pd.read_csv('Paid-Transaction.csv')
seller = pd.read_csv('Seller.csv')

print(f"Paid-Transaction shape: {paid_transaction.shape}")
print(f"Seller shape: {seller.shape}")

# Display first few rows
paid_transaction.head()

Paid-Transaction shape: (12000, 11)
Seller shape: (12000, 11)


,Paid Date,Order Number,First Name,Last Name,Meta Category,Product Name,Transaction Amount,Seller Discount,Sales Discount,Delivery Fee,Other Discount
0,20170724,"201,707,240,088,517",elvride,aries,Babies/ Kids,Pineapple Hat Anak Ala Korea - 6M - 4Y - Unise...,"300,000","153,000","10,200","9,000",0
1,20170701,"201,707,018,889,790",BASIR,Ninuk,Service/ Mokado,Pulsa BOLT 150.000,"287,800","68,000",0,0,0
2,20170707,"201,707,079,264,675",Citra,Ardi,Service/ Mokado,"XTRA Combo 12X 6GB, 12bln","35,000",0,700,0,0
3,20170720,"201,707,209,945,714",Dian,Renaldi,Fashion,Square Foldable Travel Bag / Tas Koper Luggage...,"85,000",0,"5,900",0,0
4,20170722,"201,707,220,002,354",rizal,Tamba,Gadget/ Komputer,Samsung Galaxy Note 5 Gold,"81,000",0,"5,600",0,0


## Step 2: Create "Full Name" Column

Create a new column "Full Name" that combines "First Name" and "Last Name" in title case, and insert it after the "Last Name" column.

In [2]:
# Create Full Name column with title case
paid_transaction['Full Name'] = (
    paid_transaction['First Name'].str.strip() + ' ' +
    paid_transaction['Last Name'].str.strip()
).str.title()

# Reorder columns to insert Full Name after Last Name
cols = paid_transaction.columns.tolist()
last_name_idx = cols.index('Last Name')
cols.remove('Full Name')
cols.insert(last_name_idx + 1, 'Full Name')
paid_transaction = paid_transaction[cols]

# Display sample
print("Sample Full Names:")
paid_transaction[['First Name', 'Last Name', 'Full Name']].head(10)

Sample Full Names:


,First Name,Last Name,Full Name
0,elvride,aries,Elvride Aries
1,BASIR,Ninuk,Basir Ninuk
2,Citra,Ardi,Citra Ardi
3,Dian,Renaldi,Dian Renaldi
4,rizal,Tamba,Rizal Tamba
5,fauzannor,andri,Fauzannor Andri
6,della,Suryo,Della Suryo
7,Sri,Oki,Sri Oki
8,Irene,Hana,Irene Hana
9,tofani,Valentina,Tofani Valentina


## Step 3: Calculate GMV (Gross Merchandise Value)
Add a "GMV" column after "Seller Discount".


In [3]:
# Clean numeric columns - remove commas and convert to numeric
numeric_cols = ['Transaction Amount', 'Seller Discount', 'Sales Discount',
                'Delivery Fee', 'Other Discount']

for col in numeric_cols:
    if paid_transaction[col].dtype == 'object':
        paid_transaction[col] = paid_transaction[col].astype(str).str.replace(',', '').astype(float)

# Calculate GMV
paid_transaction['GMV'] = (
    paid_transaction['Transaction Amount'] -
    paid_transaction['Seller Discount'] +
    paid_transaction['Delivery Fee']
)

# Insert GMV column after Seller Discount
cols = paid_transaction.columns.tolist()
seller_discount_idx = cols.index('Seller Discount')
cols.remove('GMV')
cols.insert(seller_discount_idx + 1, 'GMV')
paid_transaction = paid_transaction[cols]

# Display sample GMV calculations
print("Sample GMV Calculations:")
paid_transaction[['Transaction Amount', 'Seller Discount', 'Delivery Fee', 'GMV']].head(10)

Sample GMV Calculations:


,Transaction Amount,Seller Discount,Delivery Fee,GMV
0,300000.0,153000.0,9000.0,156000.0
1,287800.0,68000.0,0.0,219800.0
2,35000.0,0.0,0.0,35000.0
3,85000.0,0.0,0.0,85000.0
4,81000.0,0.0,0.0,81000.0
5,149000.0,0.0,9000.0,158000.0
6,90000.0,48100.0,0.0,41900.0
7,180000.0,96200.0,0.0,83800.0
8,300000.0,156400.0,9000.0,152600.0
9,675000.0,351900.0,10000.0,333100.0


## Step 4: Group Categories and Create Pivot Table

**Category Groups:**
- **Group 1:** Fashion, Babies/ Kids, Beauty/ Health
- **Group 2:** Service/ Mokado, Gadget/ Komputer

In [4]:
# Define category groups
category_groups = {
    'Group 1': ['Fashion', 'Babies/ Kids', 'Beauty/ Health'],
    'Group 2': ['Service/ Mokado', 'Gadget/ Komputer']
}

# Create mapping dictionary
category_to_group = {}
for group, categories in category_groups.items():
    for category in categories:
        category_to_group[category] = group

# Add Group column
paid_transaction['Group'] = paid_transaction['Meta Category'].map(category_to_group)

# Convert Paid Date to datetime
paid_transaction['Paid Date'] = pd.to_datetime(paid_transaction['Paid Date'], format='%Y%m%d')
paid_transaction['Year-Month'] = paid_transaction['Paid Date'].dt.to_period('M')

# Create pivot table
pivot_gmv = paid_transaction.pivot_table(
    values='GMV',
    index='Year-Month',
    columns='Group',
    aggfunc='sum',
    fill_value=0
)

# Add total column
pivot_gmv['Total'] = pivot_gmv.sum(axis=1)

print("Pivot Table: Total GMV per Month by Group")
pivot_gmv

Pivot Table: Total GMV per Month by Group


Group,Group 1,Group 2,Total
Year-Month,,,
2017-07,83916600.0,194045200.0,277961800.0
2017-08,59189600.0,341657100.0,400846700.0
2017-09,198279800.0,156506600.0,354786400.0
2017-10,92012400.0,135405000.0,227417400.0
2017-11,221708700.0,300590900.0,522299600.0
2017-12,89257800.0,435267100.0,524524900.0


## Step 5: Find Seller with Highest GMV in August 2017

In [8]:
# Process Seller data
for col in numeric_cols:
    if col in seller.columns and seller[col].dtype == 'object':
        seller[col] = seller[col].astype(str).str.replace(',', '').astype(float)

# Calculate GMV for seller data
seller['GMV'] = (
    seller['Transaction Amount'] -
    seller['Seller Discount'] +
    seller['Delivery Fee']
)

# Create Full Name in seller data
seller['Full Name'] = (
    seller['First Name'].str.strip() + ' ' +
    seller['Last Name'].str.strip()
).str.title()

# Convert Paid Date to datetime
seller['Paid Date'] = pd.to_datetime(seller['Paid Date'], format='%Y%m%d')

# Filter for August 2017
august_2017 = seller[
    (seller['Paid Date'].dt.year == 2017) &
    (seller['Paid Date'].dt.month == 8)
]

# Group by seller and calculate total GMV
seller_gmv_august = august_2017.groupby('Full Name')['GMV'].sum().reset_index()
seller_gmv_august = seller_gmv_august.sort_values('GMV', ascending=False)

print("Top 10 Sellers by GMV in August 2017:")
print(seller_gmv_august.head(10))

# Get the top seller
top_seller_august = seller_gmv_august.iloc[0]
print(f"ANSWER: Seller with Highest GMV in August 2017")
print(f"Seller Name: {top_seller_august['Full Name']}")
print(f"Total GMV: Rp {top_seller_august['GMV']:,.2f}")


Top 10 Sellers by GMV in August 2017:
            Full Name         GMV
1327     Petrus Sinda  18500000.0
1249  Nita Musriyatul  14006000.0
1014      Liany Ratih  14006000.0
1007      Leony Wiwit  13705000.0
1843  Wahyu Falentina  12429000.0
224        Ayda Nadia  12008000.0
631        Fifi Dinni  12008000.0
798           Ida Ika   9470000.0
1657  Supriatun Ninik   8796400.0
994         Lee Ajeng   8796400.0
ANSWER: Seller with Highest GMV in August 2017
Seller Name: Petrus Sinda
Total GMV: Rp 18,500,000.00


## Step 6: Find Seller with Most Transactions in Fashion Category (September 2017)

In [7]:
# Filter for September 2017 and Fashion category
september_2017_fashion = seller[
    (seller['Paid Date'].dt.year == 2017) &
    (seller['Paid Date'].dt.month == 9) &
    (seller['Meta Category'] == 'Fashion')
]

# Count transactions per seller
seller_count_sept = september_2017_fashion.groupby('Full Name').size().reset_index(name='Transaction Count')
seller_count_sept = seller_count_sept.sort_values('Transaction Count', ascending=False)

print("Top 10 Sellers by Transaction Count in Fashion (September 2017):")
print(seller_count_sept.head(10))

# Get the top seller
if len(seller_count_sept) > 0:
    top_seller_sept = seller_count_sept.iloc[0]
    print(f"ANSWER: Seller with Most Transactions in Fashion (Sep 2017)")
    print(f"Seller Name: {top_seller_sept['Full Name']}")
    print(f"Transaction Count: {top_seller_sept['Transaction Count']}")
else:
    print("\nNo Fashion transactions found for September 2017")

Top 10 Sellers by Transaction Count in Fashion (September 2017):
             Full Name  Transaction Count
463          Yosep Ria                  1
462        Yopi Asikin                  1
461        Yoga Albert                  1
460         Yeri Oriza                  1
459          Yekti Ani                  1
458         Yang Taqin                  1
457           Yan Dewi                  1
456  William Heriyanto                  1
455   Wiekam Sihabudin                  1
454        Wentry Moja                  1
ANSWER: Seller with Most Transactions in Fashion (Sep 2017)
Seller Name: Yosep Ria
Transaction Count: 1
